<a href="https://colab.research.google.com/github/Kirankumarpetlu/NLP/blob/main/NLPweek6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implement chart parser using CYK algorihm for the grammars

Grammers:
1. Implement Chart parser using CYK algorithm for the grammars:
* S->NP VP| NP AUXV VP
* NP-> DET Noun | PNOUN | Noun
* VP->  V  NP
* PNOUN -> JOHN
* AUXV -> is
* V-> playing
* Noun -> Game
Ex: John is playing a game




In [ ]:
import numpy as np

# Grammar in CNF
grammar = {
    'S': ['NP VP', 'NP AUXV VP'],
    'NP': ['DET NOM', 'PNOUN', 'NOUN'],
    'VP': ['VERB NP', 'V NP'],
    'PNOUN': ['JOHN'],
    'AUXV': ['Is'],
    'V': ['playing'],
    'NOUN': ['Game'],
    'DET': ['a'],
    'NOM': ['game'],
    'VERB': ['is']
}

# Reverse the grammar for easier lookup
reverse_grammar = {}
for lhs, rhs_list in grammar.items():
    for rhs in rhs_list:
        if rhs not in reverse_grammar:
            reverse_grammar[rhs] = []
        reverse_grammar[rhs].append(lhs)

def cyk_parse(sentence):
    words = sentence.split()
    n = len(words)

    # Initialize the CYK table
    table = [[set() for _ in range(n)] for _ in range(n)]

    # Fill the diagonal (length 1 substrings)
    for i in range(n):
        for lhs, rhs_list in grammar.items():
            for rhs in rhs_list:
                if words[i] == rhs:
                    table[i][i].add(lhs)

    # Fill the table for substrings of length 2 to n
    for length in range(2, n+1):
        for i in range(n - length + 1):
            j = i + length - 1
            for k in range(i, j):
                for lhs, rhs_list in grammar.items():
                    for rhs in rhs_list:
                        if len(rhs.split()) == 2:
                            B, C = rhs.split()
                            if B in table[i][k] and C in table[k+1][j]:
                                table[i][j].add(lhs)
    return 'S' in table[0][n-1]

# Test the CYK parser
sentence = "John Is playing Game"
if cyk_parse(sentence):
    print(f'The sentence "{sentence}" is valid according to the grammar.')
else:
    print(f'The sentence "{sentence}" is NOT valid according to the grammar.')

The sentence "John Is playing Game" is NOT valid according to the grammar.


In [ ]:
import nltk
from nltk import CFG
from nltk.parse import ChartParser
# Define the CFG (Context-Free Grammar)
grammar = CFG.fromstring("""
S -> NP VP
NP -> DET NOM | PNOUN
VP -> AUX V NP
DET -> 'a' | 'the'
NOM -> 'game'
PNOUN -> 'John'
AUX -> 'is'
V -> 'playing'
""")
# Create a chart parser
parser = ChartParser(grammar)
# Input sentence (must match the grammar exactly)
sentence = ["John", "is", "playing", "a", "game"]
# Parse the sentence and display output
for tree in parser.parse(sentence):
    print(tree) # Print tree structure
    tree.pretty_print() # Display tree visually # Indented this line

(S
  (NP (PNOUN John))
  (VP (AUX is) (V playing) (NP (DET a) (NOM game))))
       S                      
   ____|_____                  
  |          VP               
  |     _____|_________        
  NP   |     |         NP     
  |    |     |      ___|___    
PNOUN AUX    V    DET     NOM 
  |    |     |     |       |   
 John  is playing  a      game



2. Implement Regex parser for the given sentences.
   * The quick brown fox jumps over the lazy dog.
   * NP->DT ADJ NN
   * VP -> V NP | PP
   * P-> IN
   * V ->V
   * PP-> IN NP

In [ ]:
import re

# Grammar rules as regex patterns
grammar_patterns = {
    "DT": r"^(The|the)$",  # Determiner
    "ADJ": r"^(quick|brown|lazy)$",  # Adjective
    "NN": r"^(fox|dog)$",  # Noun
    "V": r"^(jumps|jump)$",  # Verb
    "IN": r"^(over)$",  # Preposition
}

# Grammar rules for phrases
grammar_rules = {
    "NP": r"DT ADJ NN",  # Noun Phrase: Determiner + Adjective + Noun
    "VP": r"V NP|PP",  # Verb Phrase: Verb + NP or Prepositional Phrase
    "PP": r"IN NP",  # Prepositional Phrase: Preposition + Noun Phrase
}

def tokenize(sentence):
    """Tokenize the sentence into words."""
    return sentence.split()

def match_word(word):
    """Match a word to its part of speech (POS) using regex."""
    for pos, pattern in grammar_patterns.items():
        if re.match(pattern, word):
            return pos
    return None

def parse_sentence(sentence):
    """Parse the sentence using regex patterns."""
    tokens = tokenize(sentence)
    pos_tags = [match_word(token) for token in tokens]
    print("Tokens:", tokens)
    pSSrint("POS Tags:", pos_tags)

    # Parse Noun Phrases (NP)
    np_pattern = grammar_rules["NP"].split()
    np_matches = []
    for i in range(len(pos_tags) - len(np_pattern) + 1):
        if pos_tags[i:i + len(np_pattern)] == np_pattern:
            np_matches.append((" ".join(tokens[i:i + len(np_pattern)]), "NP"))

    # Parse Prepositional Phrases (PP)
    pp_pattern = grammar_rules["PP"].split()
    pp_matches = []
    for i in range(len(pos_tags) - len(pp_pattern) + 1):
        if pos_tags[i:i + len(pp_pattern)] == pp_pattern:
            pp_matches.append((" ".join(tokens[i:i + len(pp_pattern)]), "PP"))

    # Parse Verb Phrases (VP)
    vp_matches = []
    for np_match in np_matches:
        np_text, _ = np_match
        for i in range(len(tokens)):
            if tokens[i] in grammar_patterns["V"] and tokens[i + 1:i + 1 + len(np_text.split())] == np_text.split():
                vp_matches.append((f"{tokens[i]} {np_text}", "VP"))

    # Combine all matches
    matches = np_matches + pp_matches + vp_matches
    return matches

# Test the regex parser
sentence = "The quick brown fox jumps over the lazy dog"
matches = parse_sentence(sentence)

print("\nParsed Phrases:")
for phrase, label in matches:
    print(f"{label}: {phrase}")
    #22BCE8674

Tokens: ['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
POS Tags: ['DT', 'ADJ', 'ADJ', 'NN', 'V', 'IN', 'DT', 'ADJ', 'NN']

Parsed Phrases:
NP: the lazy dog


Implement dependency parsing for:
-> I prefer the morning flight through Denver
-> The dog barked loudly at the strager

In [ ]:
import re

# Grammar rules for POS tagging
pos_rules = {
    "DET": r"^(that|this|a|the)$",
    "Noun": r"^(book|flight|meal|man|dog|stranger|morning|Denver)$",
    "Verb": r"^(prefer|barked|read|book|include)$",
    "Aux": r"^(do|does|did|can|will)$",
}

# Grammar rules for dependency parsing
grammar_rules = {
    "S": ["NP VP", "VP", "Aux NP VP"],
    "NP": ["DET NOM"],
    "NOM": ["Noun", "Noun NOM"],
    "VP": ["Verb", "Verb NP"],
}

def tokenize(sentence):
    """Tokenize the sentence into words."""
    return sentence.split()

def pos_tag(tokens):
    """Assign POS tags to tokens using regex."""
    pos_tags = []
    for token in tokens:
        for pos, pattern in pos_rules.items():
            if re.match(pattern, token):
                pos_tags.append((token, pos))
                break
        else:
            pos_tags.append((token, "UNKNOWN"))  # Unknown POS
    return pos_tags

def parse_sentence(sentence):
    """Parse the sentence and identify dependencies."""
    tokens = tokenize(sentence)
    pos_tags = pos_tag(tokens)
    print("Tokens:", tokens)
    print("POS Tags:", pos_tags)

    dependencies = []

    # Parse NP (Noun Phrase)
    for i in range(len(pos_tags)):
        token, pos = pos_tags[i]
        if pos == "DET":
            # Check if the next word is a Noun
            if i + 1 < len(pos_tags) and pos_tags[i + 1][1] == "Noun":
                dependencies.append((pos_tags[i + 1][0], "DET", token))  # Noun depends on DET

    # Parse VP (Verb Phrase)
    for i in range(len(pos_tags)):
        token, pos = pos_tags[i]
        if pos == "Verb":
            # Check if the next word is an NP
            if i + 1 < len(pos_tags) and pos_tags[i + 1][1] in ["DET", "Noun"]:
                dependencies.append((pos_tags[i + 1][0], "OBJ", token))  # NP depends on Verb as object
            # Check if the previous word is an NP
            if i - 1 >= 0 and pos_tags[i - 1][1] in ["DET", "Noun"]:
                dependencies.append((token, "SUBJ", pos_tags[i - 1][0]))  # Verb depends on NP as subject

    return dependencies

# Test the dependency parser
sentences = [
    "I prefer the morning flight through Denver",
    "The dog barked loudly at the stranger",
    "The man read this book",
]

for sentence in sentences:
    print(f"\nSentence: {sentence}")
    dependencies = parse_sentence(sentence)
    print("Dependencies:")
    for dep in dependencies:
        print(f"{dep[0]} --({dep[1]})--> {dep[2]}")
        #22BCE8674


Sentence: I prefer the morning flight through Denver
Tokens: ['I', 'prefer', 'the', 'morning', 'flight', 'through', 'Denver']
POS Tags: [('I', 'UNKNOWN'), ('prefer', 'Verb'), ('the', 'DET'), ('morning', 'Noun'), ('flight', 'Noun'), ('through', 'UNKNOWN'), ('Denver', 'Noun')]
Dependencies:
morning --(DET)--> the
the --(OBJ)--> prefer

Sentence: The dog barked loudly at the stranger
Tokens: ['The', 'dog', 'barked', 'loudly', 'at', 'the', 'stranger']
POS Tags: [('The', 'UNKNOWN'), ('dog', 'Noun'), ('barked', 'Verb'), ('loudly', 'UNKNOWN'), ('at', 'UNKNOWN'), ('the', 'DET'), ('stranger', 'Noun')]
Dependencies:
stranger --(DET)--> the
barked --(SUBJ)--> dog

Sentence: The man read this book
Tokens: ['The', 'man', 'read', 'this', 'book']
POS Tags: [('The', 'UNKNOWN'), ('man', 'Noun'), ('read', 'Verb'), ('this', 'DET'), ('book', 'Noun')]
Dependencies:
book --(DET)--> this
this --(OBJ)--> read
read --(SUBJ)--> man
